In [19]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [20]:
file_name = "../dataset/raw/master_sheet.xlsx"
sheet_name = "Sheet1"

In [21]:
xl_sheet = pd.read_excel(file_name,sheet_name=sheet_name)

In [22]:
csv_path = "../dataset/raw/master_sheet.csv"
xl_sheet.to_csv(csv_path, index=False)

In [23]:
df = pd.read_csv(csv_path)
df = df.iloc[:,1:-16]
df.head()

,ID,Age,Sex,Presentation,Admission_Date,Admission Time,father,mother,firstdeg,seconddeg,...,OnD_Anticoagulant,OnD_INSULIN,OnD_Oral_Anti_glycemics,OnD_Non_statin_lipid_drugs,30 days Follow up Done Date,Method,Follow Up Status,Reinfarction,Hospitalization for Heart Failure,Stroke
0,DT01,45,Male,Direct,2021-07-29,14:10:00,No,No,No,No,...,No,No,No,No,2021-08-31,Telephonic,Alive,No,Yes,No
1,DT02,61,Male,Referral,2021-07-29,14:40:00,No,No,No,No,...,Yes,No,Yes,No,2021-08-31,Telephonic,Alive,No,No,No
2,DT03,45,Male,Referral,2021-07-28,17:10:00,No,No,No,No,...,No,No,Yes,No,2021-08-31,Telephonic,Deceased,No,No,No
3,DT04,50,Male,Referral,2021-07-29,14:20:00,No,No,No,No,...,Yes,No,No,No,2021-08-31,Telephonic,Alive,No,No,No
4,DT05,70,Female,Direct,2021-07-27,12:30:00,No,No,No,No,...,Yes,No,Yes,No,2021-08-31,Telephonic,Alive,No,No,No


In [24]:
def replace(val):
    if isinstance(val,str):
        if(val=="No"):
            return 0
        else:
            if(val.isdigit() or val.isdecimal()):
                return float(val)
            else:
                return val
    else:
        return val

In [25]:
def convert_time(t):
    h,m,s = map(int,t.split(':'))
    return h*60 + m + s/60

In [26]:
num = ['Age', 'Number of Cigarettes Beedi per day', 'Number of years smoked', 'Height', 'Weight', 'Heart_Rate', 'BP_Systolic', 'BP_Diastolic',
        'Exact Time of Symptom Onset', 'BASELINE SERUM CREATININE', 'HAEMOGLOBIN', 'HBA1C', 'Total_cholesterol', 'LDL', 'HDL', 'Triglycerides',
        'APO B(g/l)', 'Liporotein A', 'Admission Time'
      ]

cat = ['Sex','Presentation','father','mother', 'firstdeg', 'seconddeg', 'Atrial fib flutter', 'Heart Failure', 'PriorPCI', 'Diabetes Mellitus', 'Cancer',
        'Hypertension', 'On Dialysis', 'Prior CABG', 'Hyperlipidemia', 'Prior MI', 'PAD', 'Prior CVA', 'Prior Angina', 'Physical_Activity',
        'Average_Sleep_Duration', 'Stress_Home', 'Stress_Work_Related', 'Stress_General', 'Tobacco_Use', 'Beedi', 'Cigarette', 'Hookah', 'Tobacco Chewing',
        'Multiple ', 'Alcohol', 'Gym_Supplements', 'Cannabis', 'Cocaine', 'Walking', 'ASPRIN', 'STATIN', 'P2Y12 Inhibitors', 'Beta_blocker', 'CalciumChannel',
        'ACE ARB', 'Aldosterone Blocking Antagonist', 'ARNI', 'Anticoagulant', 'INSULIN', 'Oral Anti glycemics', 'Non statin lipid drugs', 'Homeopathy',
        'Ayurveda', 'Symptoms_name', 'PRECEDING_ACTIVITY', 'CardiacStatus_Presentation', 'Time since symptoms onset to first medical contact',
        'Mode_of_transport', 'Type of First Medical Contact', 'Nature of Treatment', 'MItype_on_ECG', 'ASPIRIN', 'STATIN.1', 'P2Y12 Inhibitors.1',
        'Beta blocker', 'THROMBOLYSIS', 'RWMA', 'Trop_T', 'Coronary_angiography', 'Age 65-74/>75', 'Systolic BP<100', 'Heart Rate>100', 'KILLIP II-IV',
        'Anterior STE or LBBB', 'H/O Diabetes, HTN or Angina', 'Weight < 67 Kgs', 'Time to Treatment >4 Hours', 'EJECTION FRACTION(%)',
        'PRESENCE OF MITRAL REGURGITATION', 'LV THROMBUS', 'VSD', 'LV ANEURYSM', 'ACUTE_MR', 'Time_since_symptom_onset_to_1st_ECG'
       ]

In [27]:
df['Exact Time of Symptom Onset'] = df['Exact Time of Symptom Onset'].apply(lambda x: convert_time(x))
df['Admission Time'] = df['Admission Time'].apply(lambda x: convert_time(x))

In [28]:
for col in df.columns:
    df[col] = df[col].fillna(0)

In [29]:
for col in num:
    df[col] = df[col].apply(lambda x: replace(x))

In [30]:
target = df["Follow Up Status"]

In [31]:
df_num = df[num]
df_cat = df[cat]

df_cat = pd.get_dummies(df_cat, columns=cat, drop_first=False, dtype='int64')

In [32]:
df_num.loc[:,'Pack_Year'] = df_num['Number of Cigarettes Beedi per day']/20 + df_num['Number of years smoked']
df_num.loc[:,'BMI'] = df_num['Weight']/((df_num['Height']/100)**2)

/var/folders/fx/m8jghh2j0xn7xvzxjj0rz1sr0000gn/T/ipykernel_45320/525154266.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_num.loc[:,'Pack_Year'] = df_num['Number of Cigarettes Beedi per day']/20 + df_num['Number of years smoked']
/var/folders/fx/m8jghh2j0xn7xvzxjj0rz1sr0000gn/T/ipykernel_45320/525154266.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_num.loc[:,'BMI'] = df_num['Weight']/((df_num['Height']/100)**2)


In [33]:
df_num = pd.DataFrame(StandardScaler().fit_transform(df_num), columns=df_num.columns)

In [34]:
df = pd.concat([df_num,df_cat],axis=1)
df = pd.concat([df,target],axis=1)

In [35]:
csv_path = "../dataset/preprocessed.csv"

In [36]:
df.to_csv(csv_path, index=False)